# Harmony SMAP L2 gridding service

This notebook describes the basic functionality of the service and how to call from `harmony-py`

### Prerequisites

The dependencies for this notebook are listed in the `docs-requirements.txt` file, launch your jupyter notebook from an environment that includes those dependencies.


A `.netrc` file must also be located in the test directory of this repository with access to the UAT Earthdata environment.


To follow along, download the original input SPL2SMP file into this directory from this url: https://harmony.uat.earthdata.nasa.gov/service-results/harmony-uat-eedtest-data/C1268429309-EEDTEST/SMAP_L2_SM_P_00872_D_20150401T044811_R19240_001.h5





### The Data
There are 4 SMAP test collections configured in UAT: 

| Collection          | Short Name | Name                                                                |
|---------------------|------------|---------------------------------------------------------------------|
| C1268429712-EEDTEST | SPL2SMP_E  | SMAP Enhanced L2 Radiometer Half-Orbit 9 km EASE-Grid Soil Moisture |
| C1268429729-EEDTEST | SPL2SMA    | SMAP L2 Radar Half-Orbit 3 km EASE-Grid Soil Moisture               |
| C1268429748-EEDTEST | SPL2SMAP   | SMAP L2 Radar/Radiometer Half-Orbit 9 km EASE-Grid Soil Moisture    |
| C1268429309-EEDTEST | SPL2SMP    | SMAP L2 Radiometer Half-Orbit 36 km EASE-Grid Soil Moisture         |

Any of these collections can be used with the smap-l2-gridder.

### What is L2 Gridded data?

Level 2 Gridded Data is data that is stored in 1D arrays along with row and column indices.



| idx             |  0 |  1 |  2 |  3 |  4 |  5 |  6 |  7 | ... |
|-----------------|----|----|----|----|----|----|----|----|-----|
| data_variable   | 10 | 20 | 25 | 15 | 75 | 90 | 60 | 30 | ... |
| EASE_row_idx    |  3 |  3 |  2 |  1 |  0 |  1 |  2 |  3 | ... |
| EASE_column_idx |  3 |  2 |  2 |  1 |  1 |  0 |  0 |  0 | ... |



### Input Data Structure

Open the input Level 2 Gridded Data file and examine the top level groups

On a mac you can open the input file in panoply to examine it with the next cell.

In [ ]:
!open -a panoply SMAP_L2_SM_P_00872_D_20150401T044811_R19240_001.h5

Open with xarray and examine the groups and some data variables

In [ ]:
import xarray as xr

in_datatree = xr.open_datatree(
    'SMAP_L2_SM_P_00872_D_20150401T044811_R19240_001.h5', decode_times=False
)
groups = [c for c in in_datatree.children]
print(f'Top level Groups: {groups}\n')

data_vars = in_datatree['Soil_Moisture_Retrieval_Data'].data_vars
print('First 10 data variables:')
list(data_vars)[0:10]

##### Examine a single variable

All of the data is stored in a single 1-dimensional array 

In [ ]:
in_datatree['Soil_Moisture_Retrieval_Data']['albedo']

### Locating grid values

The L2 data is *"gridded"* by locating the correct place in the output grid for each value in input array. 

Data values are entered into the grid, by using the column and row values as indices. 

Simply: for each index `i` in the input array:

```
  row = EASE_row_index[i]
  column = EASE_column_index[i]
  grid[row,column] = data_value[i]
```

In [ ]:
data_variable = [10, 20, 25, 15, 75, 90, 60, 30]  # ...
EASE_row_index = [3, 3, 2, 1, 0, 1, 2]  # ...
EASE_column_index = [3, 2, 2, 1, 1, 0, 0]  # ...

In [ ]:
import numpy as np

grid = np.zeros((4, 4))
for data, row, column in zip(data_variable, EASE_row_index, EASE_column_index):
    print(f'grid[{row}, {column}] = {data}')
    grid[row, column] = data

#### The resulting grid

In [ ]:
print(grid)


This is the essentially how Level 2 Gridded data is inflated and what is happening in the service.


## Using the service

use [harmony-py](https://github.com/nasa/harmony-py) and initialize a UAT Harmony client

In [ ]:
from harmony import Client, Collection, Environment, Request

harmony_client = Client(env=Environment.UAT)

Create a Harmony Request using a known SPL2SMA granule for testing.

The request has:
- a collection for SPL2SMA,
- a granule_id that crosses a large section of land
- a crs of WGS84 and allows harmony to know to invoke this service 
- a format to say we want netcdf output.



In [ ]:
test_request = Request(
    collection=Collection(id='C1268429309-EEDTEST'),
    granule_id='G1268454418-EEDTEST',
    crs='EPSG:4326',
    format='application/x-netcdf4',
)

Submit the request to harmony and download the results.

In [ ]:
job_id = harmony_client.submit(test_request)

for filename in [
    file_future.result()
    for file_future in harmony_client.download_all(job_id, overwrite=True)
]:
    print(f'Downloaded processed file from Harmony: {filename}')

In [ ]:
print(filename)

## Open and examine the gridded data file

again, on a mac with panopoly you can open the file with the next cell

The metadata is NetCDF-CF compliant and panoply allows you to plot any of the output gridded data.

In [ ]:
!open -a panoply {filename}

In [ ]:
output_dt = xr.open_datatree(filename, decode_times=False)

groups = [c for c in output_dt.children]
print(f'Top level Groups: {groups}\n')

data_vars = output_dt['Soil_Moisture_Retrieval_Data'].data_vars
print('First 10 data variables:')
list(data_vars)[0:10]

The data has groups and data variables like the input data file.

But now the data is 2-dimensional and for the SPL2SMAP data values, they are gridded to the EASE-Grid 2.0 36km Global Equal Area grid

In [ ]:
output_dt['Soil_Moisture_Retrieval_Data']['soil_moisture']

You can see in the `soil_moisture`'s attributes a `grid_mapping` that points to  `crs` following NetCDF-CF conventions.


Looking at the `crs` variable you can see its metadata contains all of the coordinate reference information to describe the projection.

In [ ]:
print(output_dt['Soil_Moisture_Retrieval_Data']['crs'])

### Plot some raw data

In [ ]:
import matplotlib.pyplot as plt

variable = 'soil_moisture'
soil_moisture_data = output_dt['Soil_Moisture_Retrieval_Data']['soil_moisture']
plt.imshow(soil_moisture_data)

guess column/row boundaries for the valid data from the above

In [ ]:
row_min = 0
row_max = 300
col_min = 400
col_max = 700

# Set the projected limits from

y_coord_min = soil_moisture_data['y-dim'][row_min]
y_coord_max = soil_moisture_data['y-dim'][row_max]
x_coord_min = soil_moisture_data['x-dim'][col_min]
x_coord_max = soil_moisture_data['x-dim'][col_max]

Plot an image with geographic boundaries for the region that has data.

In [ ]:
import cartopy.crs as ccrs

fig, ax = plt.subplots(subplot_kw={'projection': ccrs.epsg(6933)})
ax.set_extent([x_coord_min, x_coord_max, y_coord_min, y_coord_max], crs=ccrs.epsg(6933))

# Plot data
soil_moisture_data.plot(ax=ax, transform=ccrs.epsg(6933))
ax.set_title(f'sample plot of {variable}', pad=20, fontsize=14)

# Add map features
ax.coastlines()
ax.gridlines()

plt.show()